In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader
from collections import deque
import numpy as np
import multiprocessing
import time
from numpy import array
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(1)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
samplesize = 5

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
ppg_data = pd.read_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/ppg.csv')
ecg_data = pd.read_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/ecg.csv')
# other_data = pd.read_csv('/content/drive/My Drive/QueenslandDataset/Processed Data/filtered_hr_sys_dias.csv')
hr = pd.read_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/hr.csv')
sys = pd.read_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/sys.csv')
dias = pd.read_csv('/content/drive/My Drive/QueenslandDataset/MovingWindowdata/dias.csv')

In [ ]:
other_data

,AVG_HR,Sys,Dias
0,51.073,113.0,60.0
1,60.000,93.0,47.0
2,60.298,93.0,47.0
3,71.871,93.0,47.0
4,65.872,93.0,47.0
...,...,...,...
9482,69.504,114.0,65.0
9483,68.937,114.0,65.0
9484,67.859,114.0,65.0
9485,67.307,114.0,65.0


In [4]:
ppg_data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0,0.36850,0.36801,0.36728,0.36630,0.36508,0.36215,0.35995,0.35726,0.35458,0.35043,0.34945,0.34872,0.34847,0.34994,0.35189,0.35458,0.35775,0.36703,0.37289,0.37998,0.38706,0.40488,0.41709,0.43101,0.44664,0.47961,0.49548,0.51038,0.52357,0.54383,0.55067,0.55067,0.56801,0.59365,0.61099,0.61099,0.61099,0.61099,0.61099,...,0.52894,0.52991,0.53040,0.53065,0.53114,0.53162,0.53211,0.53236,0.53260,0.53260,0.53260,0.53236,0.53114,0.53065,0.53040,0.53016,0.53016,0.53016,0.53016,0.53016,0.53016,0.53016,0.53016,0.52967,0.52869,0.52845,0.52845,0.52869,0.52918,0.53114,0.53260,0.53358,0.53455,0.53578,0.53602,0.53602,0.53602,0.53602,0.53602,0.53602
1,1,0.40024,0.40464,0.40659,0.40708,0.40562,0.39585,0.38730,0.37705,0.36557,0.34408,0.33626,0.32869,0.31770,0.31502,0.31355,0.31331,0.31404,0.31673,0.31844,0.32063,0.32527,0.32772,0.33040,0.33333,0.33993,0.34359,0.34701,0.35043,0.35385,0.36117,0.36484,0.37411,0.37411,0.38584,0.39219,0.39219,0.39219,0.39219,0.41978,...,0.55092,0.55385,0.55507,0.55629,0.55824,0.55922,0.55995,0.56068,0.56190,0.56264,0.56313,0.56288,0.56166,0.56068,0.55946,0.55849,0.55751,0.55556,0.55482,0.55409,0.55409,0.55507,0.55580,0.55702,0.56117,0.56337,0.56532,0.56679,0.56947,0.57070,0.57167,0.57289,0.57436,0.57778,0.57949,0.58120,0.58291,0.58486,0.58486,0.58486
2,2,0.42808,0.42173,0.41563,0.41001,0.40049,0.39658,0.39292,0.38950,0.38657,0.38169,0.37998,0.37778,0.37534,0.37094,0.36923,0.36801,0.36752,0.36850,0.36947,0.37094,0.37387,0.37485,0.37534,0.37558,0.37534,0.37534,0.37558,0.37607,0.37680,0.37875,0.37998,0.37998,0.38242,0.38437,0.38535,0.38535,0.38535,0.38535,0.38559,...,0.52967,0.52894,0.52747,0.52576,0.51941,0.51380,0.50623,0.49768,0.47350,0.45812,0.44151,0.42369,0.40635,0.37509,0.35995,0.34554,0.33370,0.32430,0.31722,0.31038,0.30549,0.30110,0.30037,0.29988,0.29963,0.29963,0.29963,0.29963,0.30012,0.30183,0.30379,0.30647,0.31136,0.31502,0.31893,0.32283,0.32698,0.33358,0.33651,0.34066
3,3,0.48523,0.48303,0.48205,0.48107,0.48010,0.47766,0.47619,0.47473,0.47302,0.46960,0.46740,0.46471,0.45934,0.45641,0.45372,0.45128,0.44908,0.44542,0.44371,0.44274,0.44249,0.44396,0.44591,0.44908,0.45275,0.46056,0.46422,0.46691,0.46935,0.46935,0.46789,0.46520,0.46520,0.45690,0.44640,0.44640,0.44640,0.44640,0.42442,...,0.33333,0.33382,0.33504,0.33675,0.33822,0.33846,0.33748,0.33578,0.33455,0.33358,0.33333,0.33333,0.33333,0.33333,0.33333,0.33382,0.33675,0.33871,0.34066,0.34286,0.34554,0.35092,0.35311,0.35531,0.35751,0.36044,0.36215,0.36410,0.36801,0.36899,0.36947,0.36972,0.36899,0.36557,0.36361,0.36166,0.35971,0.35287,0.34994,0.34750
4,4,0.50379,0.50427,0.50501,0.50623,0.50769,0.51111,0.51282,0.51453,0.51575,0.51673,0.51697,0.51746,0.51893,0.52015,0.52161,0.52381,0.52650,0.53284,0.53651,0.53968,0.54237,0.54945,0.55311,0.55678,0.55995,0.56557,0.56752,0.56899,0.56972,0.56899,0.56777,0.56777,0.56337,0.55751,0.55043,0.55043,0.55043,0.55043,0.55043,...,0.48889,0.48523,0.48132,0.47741,0.47082,0.46886,0.46740,0.46642,0.46520,0.46545,0.46642,0.46764,0.46911,0.47155,0.47253,0.47326,0.47424,0.47619,0.47692,0.47766,0.47863,0.47937,0.47888,0.47766,0.47302,0.47033,0.46764,0.46520,0.46300,0.45836,0.45665,0.45568,0.45543,0.45641,0.45739,0.45910,0.46154,0.46520,0.46642,0.46740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191233,191233,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012,0.50012

In [ ]:
ecg_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.070,-0.075,-0.065,-0.055,-0.050,-0.045,-0.035,-0.040,-0.040,-0.040,-0.040,-0.045,-0.045,-0.045,-0.045,-0.045,-0.035,-0.015,0.005,0.020,0.030,0.060,0.060,0.055,0.010,0.010,-0.030,-0.030,-0.030,-0.030,-0.085,...,-0.095,-0.095,-0.090,-0.080,-0.075,-0.070,-0.070,-0.065,-0.065,-0.065,-0.060,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.060,-0.060,-0.060,-0.055,-0.035,-0.015,0.000,0.010,0.030,0.050,0.045,0.015,-0.025,-0.050,-0.075,-0.095,-0.095,-0.100,-0.105,-0.105,-0.100,-0.115,-0.155
1,-0.045,-0.040,-0.025,-0.005,0.005,0.025,0.050,0.090,0.120,0.155,0.185,0.225,0.250,0.260,0.260,0.245,0.225,0.180,0.120,0.050,-0.015,-0.075,-0.130,-0.150,-0.170,-0.170,-0.175,-0.170,-0.170,-0.170,-0.165,-0.160,-0.160,-0.160,-0.160,-0.160,-0.160,-0.160,-0.160,-0.155,...,-0.095,-0.090,-0.085,-0.080,-0.065,-0.035,-0.025,-0.005,0.030,0.040,0.030,0.000,-0.030,-0.065,-0.095,-0.115,-0.120,-0.125,-0.125,-0.120,-0.115,-0.105,-0.140,-0.155,0.015,0.565,1.140,0.950,0.310,-0.030,-0.075,-0.085,-0.105,-0.115,-0.115,-0.105,-0.100,-0.085,-0.070,-0.060
2,-0.095,-0.090,-0.085,-0.075,-0.065,-0.050,-0.040,-0.030,-0.015,-0.005,0.005,0.030,0.050,0.080,0.115,0.155,0.185,0.225,0.250,0.265,0.275,0.275,0.255,0.225,0.175,0.105,0.050,-0.010,-0.080,-0.115,-0.145,-0.145,-0.165,-0.170,-0.165,-0.165,-0.165,-0.165,-0.150,-0.145,...,-0.160,-0.160,-0.155,-0.155,-0.155,-0.150,-0.150,-0.150,-0.145,-0.145,-0.145,-0.140,-0.135,-0.135,-0.130,-0.125,-0.120,-0.105,-0.085,-0.055,-0.040,-0.025,-0.005,0.005,-0.005,-0.025,-0.060,-0.095,-0.120,-0.135,-0.140,-0.140,-0.135,-0.130,-0.120,-0.125,-0.170,-0.135,0.175,0.835
3,-0.115,-0.130,-0.130,-0.125,-0.115,-0.115,-0.115,-0.115,-0.115,-0.115,-0.120,-0.125,-0.125,-0.130,-0.125,-0.125,-0.115,-0.105,-0.100,-0.095,-0.085,-0.080,-0.075,-0.075,-0.075,-0.070,-0.065,-0.065,-0.060,-0.055,-0.045,-0.045,0.005,0.015,0.035,0.055,0.055,0.055,-0.015,-0.025,...,-0.075,-0.050,-0.040,-0.020,0.010,0.010,-0.005,-0.045,-0.085,-0.105,-0.125,-0.130,-0.130,-0.125,-0.115,-0.115,-0.105,-0.125,-0.155,-0.015,0.515,1.110,0.920,0.210,-0.020,-0.095,-0.100,-0.110,-0.115,-0.115,-0.110,-0.100,-0.085,-0.080,-0.065,-0.050,-0.035,-0.020,-0.005,0.015
4,0.045,0.075,0.115,0.145,0.185,0.225,0.240,0.255,0.255,0.245,0.215,0.160,0.110,0.040,-0.035,-0.080,-0.130,-0.155,-0.170,-0.170,-0.170,-0.175,-0.170,-0.160,-0.160,-0.155,-0.150,-0.145,-0.145,-0.145,-0.140,-0.140,-0.140,-0.135,-0.135,-0.135,-0.135,-0.135,-0.120,-0.120,...,-0.135,-0.135,-0.130,-0.125,-0.125,-0.125,-0.125,-0.120,-0.115,-0.115,-0.120,-0.115,-0.115,-0.115,-0.105,-0.095,-0.095,-0.085,-0.080,-0.075,-0.075,-0.075,-0.070,-0.065,-0.065,-0.065,-0.055,-0.040,-0.015,0.000,0.010,0.040,0.050,0.050,0.015,-0.010,-0.045,-0.075,-0.085,-0.085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9482,-0.065,-0.055,-0.045,-0.035,-0.030,-0.025,-0.020,-0.010,-0.015,-0.025,-0.040,-0.055,-0.075,-0.085,-0.090,-0.100,-0.105,-0.105,-0.105,-0.105,-0.100,-0.095,-0.085,-0.085,-0.075,-0.070,-0.070,-0.070,-0.065,-0.065,-0.065,-0.065,-0.065,-0.055,-0.035,-0.015,-0.005,0.000,0.015,0.010,...,-0.035,-0.025,-0.015,-0.005,0.005,0.010,0.020,0.020,0.005,-0.015,-0.035,-0.055,-0.065,-0.075,-0.085,-0.090,-0.095,-0.095,-0.090,-0.085,-0.085,-0.085,-0.075,-0.075,-0.065,-0.065,-0.065,-0.065,-0.060,-0.060,-0.055,-0.060,-0.060,-0.065,-0.065,-0.055,-0.035,-0.015,-0.005,0.010
9483,0.020,0.015,0.005,-0.020,-0.050,-0.080,-0.090,-0.095,-0.095,-0.090,-0.080,-0.075,-0.075,-0.075

In [ ]:
ecg_data.shift(2, axis = 1, fill_value=0.0) 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.070,-0.075,-0.065,-0.055,-0.050,-0.045,-0.035,-0.040,-0.040,-0.040,-0.040,-0.045,-0.045,-0.045,-0.045,-0.045,-0.035,-0.015,0.005,0.020,0.030,0.060,0.060,0.055,0.010,0.010,-0.030,-0.030,-0.030,...,-0.105,-0.105,-0.095,-0.095,-0.090,-0.080,-0.075,-0.070,-0.070,-0.065,-0.065,-0.065,-0.060,-0.065,-0.065,-0.065,-0.065,-0.065,-0.065,-0.060,-0.060,-0.060,-0.055,-0.035,-0.015,0.000,0.010,0.030,0.050,0.045,0.015,-0.025,-0.050,-0.075,-0.095,-0.095,-0.100,-0.105,-0.105,-0.100
1,0.0,0.0,-0.045,-0.040,-0.025,-0.005,0.005,0.025,0.050,0.090,0.120,0.155,0.185,0.225,0.250,0.260,0.260,0.245,0.225,0.180,0.120,0.050,-0.015,-0.075,-0.130,-0.150,-0.170,-0.170,-0.175,-0.170,-0.170,-0.170,-0.165,-0.160,-0.160,-0.160,-0.160,-0.160,-0.160,-0.160,...,-0.100,-0.095,-0.095,-0.090,-0.085,-0.080,-0.065,-0.035,-0.025,-0.005,0.030,0.040,0.030,0.000,-0.030,-0.065,-0.095,-0.115,-0.120,-0.125,-0.125,-0.120,-0.115,-0.105,-0.140,-0.155,0.015,0.565,1.140,0.950,0.310,-0.030,-0.075,-0.085,-0.105,-0.115,-0.115,-0.105,-0.100,-0.085
2,0.0,0.0,-0.095,-0.090,-0.085,-0.075,-0.065,-0.050,-0.040,-0.030,-0.015,-0.005,0.005,0.030,0.050,0.080,0.115,0.155,0.185,0.225,0.250,0.265,0.275,0.275,0.255,0.225,0.175,0.105,0.050,-0.010,-0.080,-0.115,-0.145,-0.145,-0.165,-0.170,-0.165,-0.165,-0.165,-0.165,...,-0.160,-0.160,-0.160,-0.160,-0.155,-0.155,-0.155,-0.150,-0.150,-0.150,-0.145,-0.145,-0.145,-0.140,-0.135,-0.135,-0.130,-0.125,-0.120,-0.105,-0.085,-0.055,-0.040,-0.025,-0.005,0.005,-0.005,-0.025,-0.060,-0.095,-0.120,-0.135,-0.140,-0.140,-0.135,-0.130,-0.120,-0.125,-0.170,-0.135
3,0.0,0.0,-0.115,-0.130,-0.130,-0.125,-0.115,-0.115,-0.115,-0.115,-0.115,-0.115,-0.120,-0.125,-0.125,-0.130,-0.125,-0.125,-0.115,-0.105,-0.100,-0.095,-0.085,-0.080,-0.075,-0.075,-0.075,-0.070,-0.065,-0.065,-0.060,-0.055,-0.045,-0.045,0.005,0.015,0.035,0.055,0.055,0.055,...,-0.115,-0.100,-0.075,-0.050,-0.040,-0.020,0.010,0.010,-0.005,-0.045,-0.085,-0.105,-0.125,-0.130,-0.130,-0.125,-0.115,-0.115,-0.105,-0.125,-0.155,-0.015,0.515,1.110,0.920,0.210,-0.020,-0.095,-0.100,-0.110,-0.115,-0.115,-0.110,-0.100,-0.085,-0.080,-0.065,-0.050,-0.035,-0.020
4,0.0,0.0,0.045,0.075,0.115,0.145,0.185,0.225,0.240,0.255,0.255,0.245,0.215,0.160,0.110,0.040,-0.035,-0.080,-0.130,-0.155,-0.170,-0.170,-0.170,-0.175,-0.170,-0.160,-0.160,-0.155,-0.150,-0.145,-0.145,-0.145,-0.140,-0.140,-0.140,-0.135,-0.135,-0.135,-0.135,-0.135,...,-0.135,-0.135,-0.135,-0.135,-0.130,-0.125,-0.125,-0.125,-0.125,-0.120,-0.115,-0.115,-0.120,-0.115,-0.115,-0.115,-0.105,-0.095,-0.095,-0.085,-0.080,-0.075,-0.075,-0.075,-0.070,-0.065,-0.065,-0.065,-0.055,-0.040,-0.015,0.000,0.010,0.040,0.050,0.050,0.015,-0.010,-0.045,-0.075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9482,0.0,0.0,-0.065,-0.055,-0.045,-0.035,-0.030,-0.025,-0.020,-0.010,-0.015,-0.025,-0.040,-0.055,-0.075,-0.085,-0.090,-0.100,-0.105,-0.105,-0.105,-0.105,-0.100,-0.095,-0.085,-0.085,-0.075,-0.070,-0.070,-0.070,-0.065,-0.065,-0.065,-0.065,-0.065,-0.055,-0.035,-0.015,-0.005,0.000,...,-0.055,-0.045,-0.035,-0.025,-0.015,-0.005,0.005,0.010,0.020,0.020,0.005,-0.015,-0.035,-0.055,-0.065,-0.075,-0.085,-0.090,-0.095,-0.095,-0.090,-0.085,-0.085,-0.085,-0.075,-0.075,-0.065,-0.065,-0.065,-0.065,-0.060,-0.060,-0.055,-0.060,-0.060,-0.065,-0.065,-0.055,-0.035,-0.015
9483,0.0,0.0,0.020,0.015,0.005,-0.020,-0.050,-0.080,-0.090,-0.095,-0.095,-0.090,-0.080,-0.075,-0.075,-0.075,-0.075,-0.075,-0.065,

In [ ]:
np.hstack((ecg_data.iloc[[0]].to_numpy().T,np.full((1000,1), other_data.iloc[[0]].to_numpy()[0][0], dtype=float))).shape

(1000, 2)

In [ ]:
other_data.iloc[[0]].to_numpy()[0][0]

51.073

In [ ]:
np.full((1000,1), other_data.iloc[[0]].to_numpy()[0][2], dtype=float)

array([[60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
       [60.],
      

In [5]:
def prepareData(ppg, ecg):
    if ppg.shape[0] != ecg.shape[0]:
        raise Exception("Data shape doesn't match")
    count = ppg.shape[0]
    train_data, labels = [], []
    for i in range(count):
        offset = random.randint(-5,5)
        # print((array(ecg.iloc[[i]].to_numpy().T)))
        # train_data.append(torch.FloatTensor(np.hstack((ecg.iloc[[i]].to_numpy().T,ppg.iloc[[i]].shift(offset, axis = 1, fill_value=0.0).to_numpy().T,np.full((1000,1), other_data.iloc[[i]].to_numpy()[0][0], dtype=float),np.full((1000,1), other_data.iloc[[i]].to_numpy()[0][1], dtype=float),np.full((1000,1), other_data.iloc[[i]].to_numpy()[0][2], dtype=float)))))
        # train_data.append(torch.FloatTensor(np.hstack((ecg.iloc[[i]].to_numpy().T,ppg.iloc[[i]].shift(offset, axis = 1, fill_value=0.0).to_numpy().T,np.full((1000,1), other_data.iloc[[i]].to_numpy()[0][1], dtype=float),np.full((1000,1), other_data.iloc[[i]].to_numpy()[0][2], dtype=float)))))
        train_data.append(torch.FloatTensor(np.hstack((ecg.iloc[[i]].to_numpy().T,ppg.iloc[[i]].shift(offset, axis = 1, fill_value=0.0).to_numpy().T,sys.iloc[[i]].to_numpy().T,dias.iloc[[i]].to_numpy().T))))
        
        labels.append(torch.FloatTensor([offset]))
    train_length =  8500
    return train_data[:train_length], labels[:train_length], train_data[train_length:], labels[train_length:]
train_data, train_label, test_data, test_label = prepareData(ppg_data, ecg_data)

In [5]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, data, labels):
        self.labels = labels
        self.data = data

  def __len__(self):
        return len(self.data)

  def __getitem__(self, index):
        X = self.data[index]
        y = self.labels[index]

        return X, y

In [ ]:
train_label

[tensor([-17.]),
 tensor([-6.]),
 tensor([21.]),
 tensor([33.]),
 tensor([1.]),
 tensor([-11.]),
 tensor([33.]),
 tensor([10.]),
 tensor([-37.]),
 tensor([-23.]),
 tensor([-21.]),
 tensor([-16.]),
 tensor([11.]),
 tensor([1.]),
 tensor([29.]),
 tensor([22.]),
 tensor([-3.]),
 tensor([-40.]),
 tensor([3.]),
 tensor([-13.]),
 tensor([-16.]),
 tensor([5.]),
 tensor([-10.]),
 tensor([-46.]),
 tensor([45.]),
 tensor([28.]),
 tensor([13.]),
 tensor([-2.]),
 tensor([-15.]),
 tensor([30.]),
 tensor([23.]),
 tensor([-4.]),
 tensor([24.]),
 tensor([-4.]),
 tensor([26.]),
 tensor([-35.]),
 tensor([7.]),
 tensor([-35.]),
 tensor([-8.]),
 tensor([-16.]),
 tensor([44.]),
 tensor([13.]),
 tensor([-24.]),
 tensor([34.]),
 tensor([-23.]),
 tensor([-47.]),
 tensor([29.]),
 tensor([-23.]),
 tensor([-47.]),
 tensor([10.]),
 tensor([21.]),
 tensor([-30.]),
 tensor([-9.]),
 tensor([34.]),
 tensor([6.]),
 tensor([9.]),
 tensor([-2.]),
 tensor([10.]),
 tensor([-18.]),
 tensor([26.]),
 tensor([15.]),
 tensor([

In [7]:
#prepare data first time
traindataset = CustomDataset(train_data,train_label)
# val_data = CustomDataset(test_input,test_label)
testdataset = CustomDataset(test_data,test_label)


In [8]:
torch.save(traindataset, '/content/drive/My Drive/QueenslandDataset/Processed Data/train_data_with_moving_data_4_input.pt')

In [9]:
torch.save(testdataset, '/content/drive/My Drive/QueenslandDataset/Processed Data/test_data_with_moving_data_4_input.pt')

In [ ]:
traindataset.__getitem__(0)

(tensor([[-6.5000e-02,  4.8547e-01,  5.1073e+01,  1.1300e+02,  6.0000e+01],
         [-6.5000e-02,  4.7961e-01,  5.1073e+01,  1.1300e+02,  6.0000e+01],
         [-6.5000e-02,  4.7961e-01,  5.1073e+01,  1.1300e+02,  6.0000e+01],
         ...,
         [-1.0000e-01,  0.0000e+00,  5.1073e+01,  1.1300e+02,  6.0000e+01],
         [-1.1500e-01,  0.0000e+00,  5.1073e+01,  1.1300e+02,  6.0000e+01],
         [-1.5500e-01,  0.0000e+00,  5.1073e+01,  1.1300e+02,  6.0000e+01]]),
 tensor([-29.]))

In [ ]:
traindataset.__getitem__(0)[0].shape

torch.Size([1000, 5])

In [ ]:
#load from drive
traindataset = torch.load('/content/drive/My Drive/QueenslandDataset/Processed Data/train_data_with_other_data.pt')
testdataset = torch.load('/content/drive/My Drive/QueenslandDataset/Processed Data/test_data_with_other_data.pt')


In [10]:
train_batch_size = 168
test_batch_size = 14
train_loader = DataLoader(traindataset, shuffle=True, batch_size=train_batch_size)
test_loader = DataLoader(testdataset, shuffle=True, batch_size=test_batch_size)

In [ ]:
testdataset.__len__()

987

In [ ]:
train_data.__len__()

1000

In [11]:
class LSTM(nn.Module):
    def __init__(self, input_size=4, output_size = 1, hidden_dim = 200, n_layers = 3, drop_prob=0):
        super(LSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        # self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        # x = x.long()
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # lstm_out = self.dropout(lstm_out)
        out = self.fc(lstm_out)
        # out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        # print(out.shape)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [ ]:
model = LSTM()
model.to(device)

lr = 0.0001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [ ]:
epochs = 150
counter = 0
print_every = 100
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    # print(batch_size)
    print("Epoch {}/{}".format(i+1, epochs))
    print('-' * 20)
    epoch_loss = 0.0
    h = model.init_hidden(train_batch_size)
    model.zero_grad()
    for inputs, labels in train_loader:
        # print(inputs.shape)
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)

        output, h = model(inputs, h)
        # print("a",output.shape)
        # print("b",labels.squeeze().shape)
        optimizer.zero_grad()
        loss = criterion(output, labels.squeeze())
        # print(output)
        # print(labels)
        # break
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += output.shape[0] * loss.item()
        if counter%print_every == 0:
        #     # print(batch_size)
        #     val_h = model.init_hidden(batch_size)
        #     val_losses = []
        #     model.eval()
        #     for inp, lab in val_loader:
        #         val_h = tuple([each.data for each in val_h])
        #         inp, lab = inp.to(device), lab.to(device)
        #         out, val_h = model(inp, val_h)
        #         val_loss = criterion(out, lab.squeeze())
        #         val_losses.append(val_loss.item())
                
            # model.train()
            print("Epoch: {}/{}".format(i+1, epochs),
                  "Step: {}".format(counter),
                  "Loss: {:.6f}".format(loss.item()))
            #       "Val Loss: {:.6f}".format(np.mean(val_losses)))
            # if np.mean(val_losses) <= valid_loss_min:
            #     # torch.save(model.state_dict(), './state_dict.pt')
            #     print('Validation loss decreased ({:.6f} --> {:.6f}).'.format(valid_loss_min,np.mean(val_losses)))
            #     valid_loss_min = np.mean(val_losses)
    # break
    print("Epoch {} Loss {}".format(i+1, epoch_loss / traindataset.__len__()))

In [ ]:
#saving model to drive

torch.save(model.state_dict(), f'/content/drive/My Drive/DummyGaussianData/model_with_5_input.pt')

In [ ]:
#loading model from drive
model = LSTM(samplesize*2)
model.to(device)
model.load_state_dict(torch.load('/content/drive/My Drive/DummyGaussianData/model.pt'))
lr = 0.0001
criterion = nn.MSELoss()

In [ ]:
test_losses = []
num_correct = 0
h = model.init_hidden(test_batch_size)
counter = 0
model.eval()
for inputs, labels in test_loader:
    counter+=1
    print(f'Starting Batch {counter}')
    print('-' * 20)
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    # print(inputs)
    output, h = model(inputs, h)
    test_loss = criterion(output, labels.squeeze())
    test_losses.append(test_loss.item())
    pred = torch.round(output)  # Rounds the output to 0/1
    print(f" Offset {labels.squeeze().data},\n Prediction {output.data},\n Prediction rounded {pred}")
    correct_tensor = pred.eq(labels.squeeze().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))

Starting Batch 1
--------------------
 Offset tensor([ 23., -41.,  47.,   9.,  39., -46.,  22.,  41.,  25., -41., -38., -35.,
         10.,   2., -11., -12.,  32., -47.,  11., -38.,  -4.,  49., -30.,  36.,
        -33., -25.,   6.,  41.,  10., -36., -33.,  43.,  47.,  -8.,  12.,  14.,
        -50.,  29., -14.,  12.,  25., -42.,   1.,  -9.,  42.,  30.,  43., -36.,
        -41., -44.,  13., -34.,   3., -43.,  26.,   1., -49.,  -4.,  48.,   7.,
         15.,  -6.,  20.,   8.,  31., -30.,  10.,  27., -50.,  49., -39., -23.,
        -46., -43., -43.,   9.,  39.,  11.,  46.,   8., -35.,  10., -12., -48.,
         14.,  48.,  15.,  -8.,  28.,  19., -19.,   1., -50.,  20.,   9.,   2.,
        -34.,   6.,  49.,  18.,  46.,   0.,  43., -26.,  15., -35.,  47., -47.,
         -7.,  39.,   8.,  45.,  49., -46.,  44., -32., -50.,  13., -39.,  -8.,
          1., -11., -28., -22., -29., -49., -32., -44.,  -3.,  20.,   7.,  21.,
         20.,  28.,  25.,  37.,  16., -16.,   0., -35., -32.,   9.,  22., 